In [1]:
!pip install torch==2.5.1

In [2]:
!pip install -r ../requirements.txt

In [3]:
"""
카테고리 분류용 Llama-DNA 8B 모델 로드 & 추론 스크립트
- GPU + bitsandbytes(nf4) 지원 시: 4-bit 양자화 로드
- 그 외 환경(Windows CPU 등): 일반 float32/bfloat16 로드
"""

import os
import difflib
import random
import warnings

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# (선택) tqdm IProgress 경고 끄기
warnings.filterwarnings("ignore", category=UserWarning, module="tqdm")

# ---------------------------------------------------------------------
# 1. 하드웨어 & 라이브러리 호환성 감지
# ---------------------------------------------------------------------
USE_4BIT = False
bnb_config = None
try:
    # bitsandbytes & GPU 가 모두 준비된 경우에만 4-bit 사용
    from transformers import BitsAndBytesConfig
    import bitsandbytes as bnb

    if torch.cuda.is_available():  # GPU 존재
        # GPU compute capability(예: (8,0))가 7.x 이상이면 안전
        major_cc, _ = torch.cuda.get_device_capability(0)
        if major_cc >= 7:
            USE_4BIT = True
except (ImportError, RuntimeError, AttributeError, ValueError):
    # bitsandbytes 미설치 또는 CPU-only -> 4bit 비활성
    USE_4BIT = False

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16   # bf16 계산
    )

# ---------------------------------------------------------------------
# 2. 모델 및 토크나이저 로드
# ---------------------------------------------------------------------
MODEL_NAME = "dnotitia/Llama-DNA-1.0-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_kwargs = {
    "device_map": "auto" if torch.cuda.is_available() else {"": "cpu"}
}
if bnb_config is not None:
    model_kwargs["quantization_config"] = bnb_config

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, **model_kwargs)

# ---------------------------------------------------------------------
# 3. 카테고리 정의
# ---------------------------------------------------------------------
categories = {
    0: "졸업요건",
    1: "학교 공지사항",
    2: "학사일정",
    3: "식단 안내",
    4: "통학/셔틀 버스"
}

# 예시 문장(간단한 룰-베이스 fallback용)
category_examples = {
    0: "졸업 요건, 졸업 학점, 졸업 논문, 졸업 자격, 유예 신청",
    1: "학교 공지사항, 안내문, 휴강 안내, 장학금 공지, 긴급 알림",
    2: "수강 신청, 시험 기간, 성적 발표, 개강일, 종강일, 학사 일정",
    3: "학식, 학생 식당, 식단표, 오늘 메뉴, 급식 시간",
    4: "셔틀버스, 통학버스, 버스 시간표, 노선, 정류장 위치"
}

# ---------------------------------------------------------------------
# 4. 분류 함수
# ---------------------------------------------------------------------
def classify_query(user_query: str):
    """
    • LLM + 프롬프트를 이용해 5개 카테고리 중 하나 예측
    • 파싱 실패 시 간단한 문자열 유사도로 fallback
    • 절대 실패하지 않음 (항상 0~4 반환)
    """
    prompt = f"""다음은 사용자 질문과 관련된 카테고리 목록입니다:
0: 졸업요건
1: 학교 공지사항
2: 학사일정
3: 식단 안내
4: 통학/셔틀 버스

사용자 질문: "{user_query}"
위 질문은 어떤 카테고리에 가장 적합한가요? 숫자 레이블(0~4)만 답하세요.

카테고리 번호:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=3,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=False
    )
    response_text = tokenizer.decode(
        outputs[0][inputs.input_ids.shape[1]:],
        skip_special_tokens=True
    ).strip()

    # 숫자 파싱 시도
    digits = ''.join(filter(str.isdigit, response_text))
    if digits:
        label = int(digits[0])
        if label in categories:
            return label, categories[label]

    # ----------------- fallback: 문자열 유사도 -----------------
    best_score, best_label = 0.0, 0
    for label, example in category_examples.items():
        score = difflib.SequenceMatcher(None, user_query, example).ratio()
        if score > best_score:
            best_score, best_label = score, label

    return best_label, categories[best_label]

# ---------------------------------------------------------------------
# 5. 사용 예시
# ---------------------------------------------------------------------
if __name__ == "__main__":
    q = "오늘 학식 뭐 나와?"
    lbl, lbl_txt = classify_query(q)
    print(f"입력: {q}\n예측: {lbl} ({lbl_txt})")


C:\Users\USER\anaconda3\envs\term\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.
C:\Users\USER\anaconda3\envs\term\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--dnotitia--Llama-DNA-1.0-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To 

입력: 오늘 학식 뭐 나와?
예측: 3 (식단 안내)


In [4]:
import json
from pathlib import Path
from tqdm import tqdm

# data/question directory relative to this notebook
base_dir = Path('../data/question') if Path('../data/question').exists() else Path('data/question')
output_dir = Path('../outputs') if Path('../outputs').exists() else Path('outputs')
output_dir.mkdir(parents=True, exist_ok=True)

for path in base_dir.glob('*.json'):
    with path.open('r', encoding='utf-8') as f:
        questions_data = json.load(f)
    results = []
    for item in tqdm(questions_data, desc=path.name):
        question = item['question']
        label, _ = classify_query(question)  # VARCO 모델 사용
        results.append({'question': question, 'label': label})
    out_file = output_dir / f"{path.stem}_output.json"
    with out_file.open('w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f'✅ {out_file} 저장 완료')


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

✅ ..\outputs\classifier_questions_only_result_output.json 저장 완료


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

✅ ..\outputs\randomized_korean_questions_result_output.json 저장 완료


In [5]:
MODEL_SAVE_PATH = './models/classifier'
import os
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)
print(f'Model and Tokenizer saved to {MODEL_SAVE_PATH}')


Model and Tokenizer saved to ./models/classifier
